In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import os, sys
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from tqdm import tqdm, tqdm_notebook
# sys.path.append('../../satclip/satclip/')
# from load import get_satclip
sys.path.append('../scripts/')
from loadpaths_glc import loadpaths
import data_loading_utils as dlu
import glc_prediction_utils as gpu
path_dict = loadpaths()
import h3pandas

In [5]:
for key, df in dict_dfs.items():
    print('\n----')
    print(f'{key}: {df.shape[0]} samples')
    print(df.columns)
    n_unique_samples = df['surveyId'].nunique()
    if 'test' not in key:
        n_unique_species = df['speciesId'].nunique()
    else:
        n_unique_species = 'NA'
    print(f'Unique samples: {n_unique_samples}')
    print(f'Unique species: {n_unique_species}')


----
df_train_pa: 1483637 samples
Index(['lon', 'lat', 'year', 'region', 'country', 'speciesId', 'surveyId',
       'geometry'],
      dtype='object')
Unique samples: 88987
Unique species: 5016

----
df_test_pa: 4716 samples
Index(['lon', 'lat', 'year', 'region', 'country', 'surveyId', 'geometry'], dtype='object')
Unique samples: 4716
Unique species: NA


In [6]:
tmp = dlu.get_path_sat_patch_per_survey(1400102)
tmp

('/Users/t.vanderplas/data_offline/data_geolifeclef2024/geolifeclef-2024/PA_Train_SatellitePatches_RGB/pa_train_patches_rgb/02/01/1400102.jpeg',
 '/Users/t.vanderplas/data_offline/data_geolifeclef2024/geolifeclef-2024/PA_Train_SatellitePatches_NIR/pa_train_patches_nir/02/01/1400102.jpeg')

In [7]:
tmp = dlu.load_multiple_env_raster()
tmp

,surveyId,Elevation,LandCover,Soilgrid-bdod,Soilgrid-cec,Soilgrid-cfvo,Soilgrid-clay,Soilgrid-nitrogen,Soilgrid-phh2o,Soilgrid-sand,...,Bio10,Bio11,Bio12,Bio13,Bio14,Bio15,Bio16,Bio17,Bio18,Bio19
0,212,47.0,8.0,140.0,214.0,151.0,292.0,159.0,73.0,284.0,...,2958,2816,6355,933,143,395,2282,770,770,1917
1,222,6.0,12.0,120.0,225.0,101.0,94.0,379.0,58.0,650.0,...,2890,2746,7286,792,376,201,2283,1336,2076,1653
2,243,10.0,5.0,115.0,261.0,86.0,131.0,373.0,54.0,750.0,...,2888,2759,8546,1117,372,295,2986,1438,2448,1838
3,324,116.0,14.0,118.0,179.0,117.0,225.0,230.0,57.0,238.0,...,2938,2802,10719,1163,564,197,3216,1948,1948,2926
4,333,84.0,14.0,138.0,233.0,176.0,328.0,250.0,71.0,231.0,...,2927,2792,9353,1095,503,249,3186,1638,1638,2677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88982,3919553,23.0,12.0,116.0,199.0,65.0,116.0,311.0,59.0,622.0,...,2887,2752,8000,919,408,218,2505,1425,2187,1723
88983,3919592,138.0,1.0,118.0,245.0,113.0,156.0,405.0,54.0,555.0,...,2888,2748,6932,781,314,240,2242,1214,2029,1498
88984,3919620,37.0,12.0,113.0,233.0,70.0,71.0,253.0,56.0,798.0,...,2887,2746,9745,1201,440,246,3167,1718,2489,2413
88985,3919640,131.0,1.0,130.0,162.0,99.0,55.0,234.0,50.0,803.0,...,2895,2722,7301,816,390,211,2357,1336,2357,1558


## geo

In [4]:
dict_dfs, dict_dfs_species, dict_val_species = dlu.load_metadata(create_geo=True, add_h3=False,
                                               create_validation_set=True)
# type(dict_dfs['df_test_pa'])

Dropped PO data because takes ages with geometry
Columns dropped: ['taxonRank', 'geoUncertaintyInM', 'date', 'areaInM2', 'publisher', 'month', 'day', 'region', 'dayOfYear', 'country']
Dropped 49637/1483637 duplicates in df_train_pa_species
Created validation set with 8898 entries


In [228]:
# dict_dfs['df_val_pa']
dict_dfs_species['df_val_pa_species']

,speciesId,surveyId
276,5900,729
288,9368,729
287,53,729
286,10255,729
285,10600,729
...,...,...
1483598,3067,3919620
1483597,4077,3919620
1483596,9816,3919620
1483600,540,3919620


In [231]:
tmp = dlu.load_multiple_env_raster(mode='train')
tmp

,surveyId,Elevation,LandCover,Soilgrid-bdod,Soilgrid-cec,Soilgrid-cfvo,Soilgrid-clay,Soilgrid-nitrogen,Soilgrid-phh2o,Soilgrid-sand,...,Bio10,Bio11,Bio12,Bio13,Bio14,Bio15,Bio16,Bio17,Bio18,Bio19
0,212,47.0,8.0,140.0,214.0,151.0,292.0,159.0,73.0,284.0,...,2958,2816,6355,933,143,395,2282,770,770,1917
1,222,6.0,12.0,120.0,225.0,101.0,94.0,379.0,58.0,650.0,...,2890,2746,7286,792,376,201,2283,1336,2076,1653
2,243,10.0,5.0,115.0,261.0,86.0,131.0,373.0,54.0,750.0,...,2888,2759,8546,1117,372,295,2986,1438,2448,1838
3,324,116.0,14.0,118.0,179.0,117.0,225.0,230.0,57.0,238.0,...,2938,2802,10719,1163,564,197,3216,1948,1948,2926
4,333,84.0,14.0,138.0,233.0,176.0,328.0,250.0,71.0,231.0,...,2927,2792,9353,1095,503,249,3186,1638,1638,2677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88982,3919553,23.0,12.0,116.0,199.0,65.0,116.0,311.0,59.0,622.0,...,2887,2752,8000,919,408,218,2505,1425,2187,1723
88983,3919592,138.0,1.0,118.0,245.0,113.0,156.0,405.0,54.0,555.0,...,2888,2748,6932,781,314,240,2242,1214,2029,1498
88984,3919620,37.0,12.0,113.0,233.0,70.0,71.0,253.0,56.0,798.0,...,2887,2746,9745,1201,440,246,3167,1718,2489,2413
88985,3919640,131.0,1.0,130.0,162.0,99.0,55.0,234.0,50.0,803.0,...,2895,2722,7301,816,390,211,2357,1336,2357,1558


In [15]:
val_or_test = 'test'

dict_dfs, dict_dfs_species, dict_val_species = dlu.load_metadata(create_geo=True, add_h3=False,
                                create_validation_set=True if val_or_test == 'val' else False)
tmp = gpu.predict_using_buffer(dict_dfs=dict_dfs, dict_dfs_species=dict_dfs_species,
                            save_pred=True, method=f'lc-specific', buffer_deg=0.2,
                            eval_mode=val_or_test)
if val_or_test == 'val':
    f1_score = gpu.compute_f1_score_dicts(dict_val=dict_val_species, dict_pred=tmp)
    print(f'F1 score: {f1_score}')

Dropped PO data because takes ages with geometry
Columns dropped: ['taxonRank', 'geoUncertaintyInM', 'date', 'areaInM2', 'publisher', 'month', 'day', 'region', 'dayOfYear', 'country']
Dropped 49637/1483637 duplicates in df_train_pa_species
    LandCover  count
0         0.0     25
1         1.0     15
2         2.0     18
3         4.0     22
4         5.0     16
5         8.0     20
6         9.0     16
7        10.0     19
8        11.0     14
9        12.0     17
10       13.0     16
11       14.0     17
12       15.0     11
13       16.0     12
14       17.0     13


 44%|████▎     | 2061/4716 [00:13<00:18, 146.59it/s]

No nearby points for surveyId: 1695841 (LC: 13.0)


100%|██████████| 4716/4716 [00:31<00:00, 151.05it/s]

Predictions done (4715 total). No nearby points: 1.
Saved predictions to: /Users/t.vanderplas/data_offline/data_geolifeclef2024/predictions_glc2024/GLC24_vdplasthijs_predictions-buffer-lc-0.2-lc-specific_2024-04-16_16-22.csv


In [6]:
print(f'F1 score: {f1_score}')

F1 score: 0.3022704980935337
